In [81]:
import pandas as pd
from io import StringIO
#Importing and reading data.
co_data = pd.read_csv("codata.csv")
co_data.head()

,UnixTime,Year,Month,Day,timestamp,Aphla_HTS_Value1,Aphla_HTS_Value2,Aphla_NO2_Value1,Aphla_NO2_Value2,Aphla_O3_Value1,Aphla_O3_Value2,Aphla_CO_Value1,Aphla_CO_Value2,Aphla_SO2_Value1,Aphla_SO2_Value2,Unnamed: 15
0,1600168609,2020,09,15,11:16:49,39.14473,31.29015,43.78125,1.06250,384.65625,605.62500,42.93750,1023.96880,511.98438,511.98438,NaN
1,1600168611,2020,09,15,11:16:51,39.21683,31.48699,43.76563,1.06250,369.56250,562.75000,42.93750,1023.96880,511.98438,511.98438,NaN
2,1600168613,2020,09,15,11:16:53,39.00587,31.64569,43.71875,1.06250,367.56250,522.93750,42.93750,1023.96880,511.98438,511.98438,NaN
3,1600168615,2020,09,15,11:16:55,39.11803,32.29114,43.67188,1.06250,345.06250,480.53125,42.93750,1023.96880,511.98438,511.98438,NaN
4,1600168617,2020,09,15,11:16:57,39.13138,32.65736,43.64063,1.06250,349.56250,468.87500,42.93750,1023.96880,511.98438,511.98438,NaN


In [ ]:
# Since our data is seperated by semicolons we need to do sep=";"
co_data=co_data.iloc[: ,:-1]
co_data=co_data[co_data.Year.apply(lambda x: x.isnumeric())]
co_data=co_data.iloc[:,[1,2,3,4,11,12]]
co_data1=pd.to_datetime(co_data[['Year', 'Month','Day']])
co_data1=co_data1.astype(str)
co_data2=co_data.iloc[:,[3,4,5]]
co_data= pd.concat([co_data1,co_data2 ], axis=1)
co_data.columns=['Date','Time','CO_Value1','CO_Value2']
co_data1=pd.to_datetime(co_data['Date'] + ' ' + co_data['Time'],errors ='coerce')
co_data= pd.concat([co_data1,co_data2 ], axis=1)
co_data.columns=['Date','Time','CO_Value1','CO_Value2']
co_data['Date'] = pd.to_datetime(co_data['Date'])
co_data=co_data.set_index('Date')
co_data.drop('Time',axis = 1, inplace = True)
co_data.head()

In [71]:
co_data.tail()

,CO_Value1,CO_Value2
Date,,
2020-10-19 13:15:45,301.03125,401.93750
2020-10-19 13:15:47,299.25000,402.68750
2020-10-19 13:15:49,297.25000,401.62500
2020-10-19 13:15:51,296.18750,400.09375
2020-10-19 13:15:53,300.15625,404.90625


In [72]:
co_data[co_data.CO_Value1.apply(lambda x: x.isnumeric())]
co_data

,CO_Value1,CO_Value2
Date,,
2020-09-15 11:16:49,42.93750,1023.96880
2020-09-15 11:16:51,42.93750,1023.96880
2020-09-15 11:16:53,42.93750,1023.96880
2020-09-15 11:16:55,42.93750,1023.96880
2020-09-15 11:16:57,42.93750,1023.96880
...,...,...
2020-10-19 13:15:45,301.03125,401.93750
2020-10-19 13:15:47,299.25000,402.68750
2020-10-19 13:15:49,297.25000,401.62500


In [80]:
co_data['CO_Value1']=pd.to_numeric(co_data['CO_Value1'], errors ='coerce')

data=co_data.resample('H').mean()
data=data.interpolate()
data.shape

(819, 1)

## Forecasting the first CO values in Alphasense data ##

In [26]:
from warnings import simplefilter

import numpy as np
import pandas as pd

from sktime.datasets import load_airline
#from sktime.forecasting.arima import ARIMA, AutoARIMA
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.compose import (
    EnsembleForecaster,
    ReducedRegressionForecaster,
    TransformedTargetForecaster,
)
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sktime.forecasting.model_selection import (
    ForecastingGridSearchCV,
    SlidingWindowSplitter,
    temporal_train_test_split,
)
from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.theta import ThetaForecaster
from sktime.forecasting.trend import PolynomialTrendForecaster
from sktime.performance_metrics.forecasting import sMAPE, smape_loss
#from sktime.transformations.series.detrend import Deseasonalizer, Detrender
from sktime.utils.plotting import plot_series

simplefilter("ignore", FutureWarning)
%matplotlib inline

In [27]:
co_data1=co_data.iloc[:,[0,2]]
Date=co_data1['Date'].to_list()
CO_1=co_data1['CO_Value1'].to_list()
co_data2= pd.Series(CO_1,Date)

IndexError: positional indexers are out-of-bounds